In [1]:
import sys
import getopt
import os
import numpy as np
import pandas as pd

def nfr(d_df, profile, sname):
    sp_list = list(set(list(profile.index)).intersection(set(list(d_df.index))))
    n = len(sp_list)
    corr = np.ones(shape=(n, n)) - d_df.loc[sp_list, sp_list].values
    np.fill_diagonal(corr, 0)
    a = np.array(profile.loc[sp_list, sname])
    inter_matrix = np.dot(a.reshape(len(a), 1),a.reshape(1, len(a)))
    np.fill_diagonal(inter_matrix, 0)
    td = np.sum(inter_matrix)/2
    fr = np.sum(np.multiply(inter_matrix, corr))/2
    fr_df = pd.DataFrame(np.multiply(inter_matrix, corr), index=sp_list, columns=sp_list)
    profile = profile.loc[sp_list, sname]
    if td == 0:
        return 0
    return fr/td, fr_df, profile

In [10]:
odir = '.'
top_n = 0

abd_path = '../../test_file/SAMEA728574.relative_abundance.tsv'
d_path = '../../sp_d.tsv'
top_n = 30

if not os.path.exists(odir):
    os.makedirs(odir)

abd = pd.read_csv(abd_path, sep='\t', header=0, index_col=0)
sname  = abd.columns[0]
level = 's'
take_sp_list = []
rename_dict = {}
for idx in abd.index:
    last_l = idx.split('|')[-1]
    if last_l[0] == level:
        take_sp_list.append(idx)
        rename_dict[idx] = last_l
abd = abd.loc[take_sp_list, ]
abd.rename(index=rename_dict, inplace=True)
abd = abd.div(abd.sum(axis=0))

d_df = pd.read_csv(d_path, sep='\t', index_col=0, header=0)
nfr_df = pd.DataFrame(index=abd.columns, columns=['nFR'])

nFR, fr_df, profile = nfr(d_df, abd, sname)
nfr_df.loc[sname, 'nFR'] = nFR
row_index, col_index = np.tril_indices(len(fr_df), k=0)
fr_df.values[row_index, col_index] = 0
edge_df = fr_df.stack().reset_index()
edge_df = edge_df[edge_df[0] != 0]
edge_df.sort_values(by=0, ascending=False, inplace=True)
if top_n >= 1:
    edge_df = edge_df.head(int(top_n))
    top_n = '{:.0f}'.format(top_n)
elif top_n > 0:
    edge_df = edge_df.head(int(top_n*len(edge_df)))
else:
    top_n = 'all'

with open(os.path.join(odir, 'output.FR.nFR.txt'), 'w') as f:
    f.write(str(nFR))
edge_df.columns = ['species1', 'species2', 'weight']
edge_df.to_csv(os.path.join(odir, 'output.FR.top_edges.tsv'.format(sname, top_n)), sep='\t', index=False)


In [9]:
edge_df

,species1,species2,weight
800462,s__Phocaeicola_plebeius_A,s__Phocaeicola_coprocola,0.002055
180993,s__Prevotellamassilia_sp900540885,s__Phocaeicola_plebeius_A,0.001370
182090,s__Prevotellamassilia_sp900540885,s__Phocaeicola_coprocola,0.001368
800530,s__Phocaeicola_plebeius_A,s__Phocaeicola_dorei,0.001084
3341182,s__Phocaeicola_coprocola,s__Phocaeicola_dorei,0.001007
800794,s__Phocaeicola_plebeius_A,s__Phocaeicola_sp000434735,0.000934
3341446,s__Phocaeicola_coprocola,s__Phocaeicola_sp000434735,0.000908
800555,s__Phocaeicola_plebeius_A,s__Coprococcus_sp000154245,0.000748
3341207,s__Phocaeicola_coprocola,s__Coprococcus_sp000154245,0.000744
182422,s__Prevotellamassilia_sp900540885,s__Phocaeicola_sp000434735,0.000716
